In [11]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn, optim
import torch.nn.functional as F

In [12]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [13]:
def train(model, dataloader, epochs):
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    model.to(device)

    # train the model
    epochs = epochs
    steps = 0
    running_loss = 0

    for e in range(epochs):
        model.train()
        accuracy=0
        running_loss = 0
        for images, labels in dataloader:
            steps += 1

            # Move input and label tensors to the GPU
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

In [14]:
def eval(dataLoader, model):  
    outputs = torch.zeros(0, dtype=torch.long).to(device)
    model.to(device)
    # Model in test mode, dropout is off
    model.eval()
    result=[]
    for images, labels in dataLoader:
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
    return outputs

In [15]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, ), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
device =  torch.device("cuda:0""cuda:0" if torch.cuda.is_available() else "cpu")
num_teachers = 100
train_len = len(mnist_trainset)
test_len = len(mnist_testset)
train_len, test_len

(60000, 10000)

In [17]:
batch_size = 32
epochs=1
print_every=120
split= int(train_len/num_teachers)
split_ini = split
indices = list(range(train_len))
init=0
samples = []
teachers = []

for i in range(num_teachers):     
    t_idx = indices[init:split]
    t_sampler = SubsetRandomSampler(t_idx)
    samples.append(t_sampler)
    init = split
    split = split+split_ini
    
for sample in samples:
    loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=32, sampler=sample)
    teacher = train(Classifier(), loader, epochs)
    teachers.append(teacher)

teachers

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
preds = torch.torch.zeros((num_teachers, test_len), dtype=torch.long)
for key, teacher in enumerate(teachers):
    test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=32)
    preds[key] = eval(test_loader, Classifier())
    
preds = preds.numpy()
preds.shape, type(preds)

In [ ]:
epsilon = 0.1
def get_labels(preds, epsilon):
    beta = 1 / epsilon
    labels = np.array([]).astype(int)
    for pred in np.transpose(preds):
        label_counts = np.bincount(pred, minlength=10)
        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)
        
        new_label = np.argmax(label_counts)
        labels = np.append(labels, new_label)
    return labels

In [ ]:
labels = get_labels(preds, epsilon)
labels.shape, type(labels)

In [ ]:
from syft.frameworks.torch.differential_privacy import pate

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=labels, noise_eps=epsilon, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)